In [1]:
import numpy as np

In [7]:
data = np.loadtxt('data.csv', delimiter=',')
labels = np.loadtxt('data-labels.csv', delimiter=',', dtype=np.int)
theta1 = np.loadtxt('theta1.csv', delimiter=',').T
theta2 = np.loadtxt('theta2.csv', delimiter=',').T

In [3]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))
def unpack(theta):
    return theta[:-1], theta[-1]
def neuron_forward(data, theta, activate=sigmoid):
    '''
    Only to fulfill the requirement of question 1.
    The function is not going to be called because computation of neural networks can be further verctorized.
    '''
    weight, bias = unpack(theta)
    return activate(np.dot(data, weight) + bias)
def network_forward(data, theta1, theta2, activate=sigmoid):
    weight1, bias1 = unpack(theta1)
    activations = activate(np.dot(data, weight1) + bias1)
    weight2, bias2 = unpack(theta2)
    scores = np.dot(activations, weight2) + bias2
    return scores
def predict(scores):
    return np.argmax(scores, axis=1)
def errors(predictions, labels):
    return np.count_nonzero(predictions - labels)
def error_rate(predictions, labels):
    return errors(predictions, labels) / len(labels)

In [4]:
scores = network_forward(data, theta1, theta2)
predictions = predict(scores)
error_rate(predictions, labels)

0.826

In [5]:
def nll_loss(scores, labels):
    N, D = scores.shape
    normalized_scores = scores - np.max(scores, axis=1).reshape((N, 1))
    exponents = np.exp(normalized_scores)
    probabilities = exponents[labels] / np.sum(exponents, axis=1).reshape((N, 1))
    loss = np.mean(-np.log(probabilities))
    return loss

In [8]:
nll_loss(scores, labels)

4.2805966503395654

In [ ]:
def d_nll_loss():